# HOLIDAY_WEEK

**Goal:**
From observations we know that during holiday weeks, there is a surge in the passenger traffic and airlines add extra capacity to meet this additional demand.

**Hypothesis:**
The number of air travel passengers has been reaching all time highs for the past few years, and in an effort to maximize potiential revenue, airlines have been known to try to fit more flights into their schedule. This is especially true during holidays(weekends around holidays).
Our estimate is that this added capacity results in additional overhead due to increased network congestion.

In [0]:
#Imports

from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext, Window
from pyspark.sql.functions import concat, col, hour, minute, lpad, rpad, substring, year, month, dayofmonth, lit, to_timestamp, expr, count, when
import numpy as np

import pandas as pd, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
from datetime import datetime as dt

sqlContext = SQLContext(sc)

In [0]:
holiday_week_beg = ['2014-12-26', '2015-01-16', '2015-05-22', '2015-07-02', '2015-09-04', '2015-11-06', '2015-11-20', '2015-12-18', '2015-12-31', '2016-01-15', '2016-05-27', '2016-07-01',
                    '2016-09-02', '2016-11-10', '2016-11-18', '2016-12-23', '2017-01-13', '2017-05-26', '2017-06-30', '2017-09-01', '2017-11-03', '2017-11-17', '2017-12-22',
                    '2018-01-12', '2018-05-25', '2018-06-29', '2018-08-31', '2018-11-09', '2018-11-30', '2018-12-21', '2019-01-18', '2019-05-24',
                    '2019-06-28', '2019-08-30', '2019-11-08', '2019-11-22', '2019-12-20']

holiday_week_end = ['2015-01-04', '2015-01-25', '2015-05-31', '2015-07-05', '2015-09-13', '2015-11-15', '2015-11-29', '2015-12-27', '2016-01-03', '2016-01-24', '2016-06-05', '2016-07-10',
                    '2016-09-11', '2016-11-13', '2016-11-27', '2017-01-08', '2017-01-22', '2017-06-04', '2017-07-09', '2017-09-10', '2017-11-12', '2017-11-26', '2018-01-07', 
                    '2018-01-21', '2018-06-03', '2018-07-08', '2018-09-09', '2018-11-25', '2018-12-09', '2019-01-06', '2019-01-27', '2019-06-02',
                    '2019-07-07', '2019-09-08', '2019-11-17', '2019-12-01', '2019-12-31']

holiday_week_beg_end = sqlContext.createDataFrame(zip(holiday_week_beg, holiday_week_end), schema=['HOLIDAY_WEEK_BEGIN', 'HOLIDAY_WEEK_END'])
display(holiday_week_beg_end)

HOLIDAY_WEEK_BEGIN,HOLIDAY_WEEK_END
2014-12-26,2015-01-04
2015-01-16,2015-01-25
2015-05-22,2015-05-31
2015-07-02,2015-07-05
2015-09-04,2015-09-13
2015-11-06,2015-11-15
2015-11-20,2015-11-29
2015-12-18,2015-12-27
2015-12-31,2016-01-03
2016-01-15,2016-01-24


In [0]:
holiday_week_beg_end.registerTempTable('holiday_week_beg_end')

holiday_week_beg_end = spark.sql("""
SELECT CAST(HOLIDAY_WEEK_BEGIN as TIMESTAMP) as HOLIDAY_WEEK_BEGIN, CAST(HOLIDAY_WEEK_END as TIMESTAMP) + INTERVAL 24 HOURS as HOLIDAY_WEEK_END
FROM holiday_week_beg_end
""")

display(holiday_week_beg_end)

HOLIDAY_WEEK_BEGIN,HOLIDAY_WEEK_END
2014-12-26T00:00:00.000+0000,2015-01-05T00:00:00.000+0000
2015-01-16T00:00:00.000+0000,2015-01-26T00:00:00.000+0000
2015-05-22T00:00:00.000+0000,2015-06-01T00:00:00.000+0000
2015-07-02T00:00:00.000+0000,2015-07-06T00:00:00.000+0000
2015-09-04T00:00:00.000+0000,2015-09-14T00:00:00.000+0000
2015-11-06T00:00:00.000+0000,2015-11-16T00:00:00.000+0000
2015-11-20T00:00:00.000+0000,2015-11-30T00:00:00.000+0000
2015-12-18T00:00:00.000+0000,2015-12-28T00:00:00.000+0000
2015-12-31T00:00:00.000+0000,2016-01-04T00:00:00.000+0000
2016-01-15T00:00:00.000+0000,2016-01-25T00:00:00.000+0000


In [0]:
holiday_week_beg_end.write.parquet("dbfs:/mnt/mids-w261/team20SSDK/strategy/helpers/holiday_week_beg_end")